# Make

This notebook contains code to covert ipynb notes to a renderable website which is hosted on github pages. You can see the final outcome of this project at [sunitdeshpande.github.io/notes](https://sunitdeshpande.github.io/notes/)

# Introduction

This make notebook contains instruction for installation of the project and all code to convert the ipynb notebook to a full renderable HTML website.

## Installation

Following the following steps to install the project.

1. Create a virtual environment with conda ```conda create --name <some_name> --file=environment.yml```

## Project Structure

 - ```dist``` - Auto generated folder, contains the final HTML version of notes.
 - ```notes``` - All the notes must be stored in this folder.
 - ```static``` - Static assets of the project like images, stylesheet, javascripts etc.
 - ```template``` - HTML template for index and notes page as well has template for HTML converter must be stored here.
 - ```index.html``` - Auto generate index page of the website.
 

## Creating Notes

Notes must be created under ```notes``` directory. Under ```notes``` directory any folder structure can be followed. The final HTML file to be generate will be created with same parents folder. So the folder structure must match the URL pattern that is expected for the note. The code below will find any ```.ipynb``` file and use it as a candidate to convert to HTML.

Each note must contain the following metadata which will be used in the final conversion process.

```
{
    "notes_info": {
        "title": "Title of the note",
        "description": "Description of the note",
        "image": "Image of the note to shown",
        "slug": "(Optional) Slug of the note to be used as HTML filename"
    }
}
```

After creating the note. Run the code below in sequence and a ```dist``` folder must be created with all notes converted to html in it and also ```index.html``` will be create in the projects root directory.

# Code

Code to convert .ipynb file to static html website.


## Cleaner

Cleans auto generated directories

In [2]:
import shutil

class Cleaner(object):

    __DELETE_LIST = [
        'dist',
        'index.html'
    ]

    @staticmethod
    def clean():
        [
            shutil.rmtree(path) if os.path.isdir(path) else os.remove(path)
            for path in Cleaner.__DELETE_LIST 
            if os.path.exists(path)
        ]

## Code Highlighter Filter

Jinja 2 filter to syntax highlight code block

In [3]:
from pygments import highlight
from pygments.formatters import HtmlFormatter

from pygments.lexers.jvm import JavaLexer
from pygments.lexers.javascript import TypeScriptLexer
from pygments.lexers import PythonLexer

class CodeHighlighterFilter(object):

    __KERNEL_LEXER_MAPPING = {
        'scijava': JavaLexer,
        'typescript': TypeScriptLexer
    }
    
    __DEFAULT_LEXER_CLASS = PythonLexer

    def __call__(self, source, *args, **kwargs):
        
        lexer_class = self.__KERNEL_LEXER_MAPPING.get(kwargs.get('metadata', {}).get('kernel', None), self.__DEFAULT_LEXER_CLASS)
        
        return highlight(source, 
                        lexer_class(),
                        HtmlFormatter())

## Coverter

Convert is a class which converts ipynb to other formats. The follow of the convert is as follows.

* Create appropriate folder under **dist** directory, if not exists.
* Covert .ipynb files to other format


In [13]:
import os
import re
import shutil
import glob

from bs4 import BeautifulSoup
from jinja2 import Template

import nbconvert
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor


class HTMLConverter(object):

    __STATIC_ASSETS_PREFIX = '/notes'
    
    __PATH_NOTEBOOK_TEMPLATE = 'template/note.tpl'
    __PATH_NOTE_TEMPLATE = 'template/note.html'
    __PATH_INDEX_TEMPLATE = 'template/index.html'
    __PATH_NOTES_FOLDER = 'notes'
    __PATH_STATIC_FOLDER = 'static'
    __PATH_OUTPUT_FOLDER = 'dist'    

    __OUTPUT_FOLDER_PREFIX = 'html'
    __OUTPUT_INDEX_FILE_PATH = 'dist/index.html'

    __METADATA_NOTEBOOK_KEY = 'note_info'

    __html_exporter = None
    __execute_preprocessor = None;


    @property
    def html_exporter(self):
        # Create singletion html exporter
        if self.__html_exporter is None:        
            self.__html_exporter = nbconvert.HTMLExporter()
            self.__html_exporter.template_file = self.__PATH_NOTEBOOK_TEMPLATE
            self.__html_exporter.filters['highlight_code'] = CodeHighlighterFilter()

        return self.__html_exporter


    def convert(self):
        generated_notes = []

        for filepath in self.__get_all_ipynb_files():

            outputdir_path = self.__create_output_dir(filepath)
            
            notebook, metadata = self.__read_ipynb_file(filepath)
            
            body, resources = self.__convert_ipynb_notebook(notebook)
            
            output_file = self.__write_output_file(outputdir_path, metadata, body, resources)

            generated_notes.append({
                'html_file': output_file,
                'title': metadata['title'],
                'image': metadata['image'] if metadata['image'].startswith('/') else '/{}'.format(metadata['image']),
                'description': metadata['description']
            })
        
        # Copy static folder
        self.__copy_static_folder()

        # Create index file
        self.__write_index_file(generated_notes)


    def __get_all_ipynb_files(self):
        return [
            filepath 
            for filepath in glob.iglob('{rootdir}/**/*.ipynb'.format(rootdir=self.__PATH_NOTES_FOLDER), recursive=True)
        ]

    def __create_output_dir(self, filepath):

        # Make the outputdir path
        output_directory = os.path.dirname(os.path.join(self.__PATH_OUTPUT_FOLDER, self.__OUTPUT_FOLDER_PREFIX, filepath))

        # Create output directory if it not exists
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        return output_directory

    def __read_ipynb_file(self, inputfile):
        # Get file data
        with open(inputfile, 'r') as file:
            data = file.read()

        # Initialize the notebook
        notebook = nbformat.reads(data, as_version=4)

        # Get metadata
        metadata = notebook['metadata'][self.__METADATA_NOTEBOOK_KEY]
        metadata['filename'] = os.path.basename(inputfile) 

        return notebook, metadata

    def __copy_static_folder(self):
        output_folder = os.path.join(self.__PATH_OUTPUT_FOLDER, 'static');
        input_folder = self.__PATH_STATIC_FOLDER
        
        # Delete old folder
        if (os.path.exists(output_folder)):
            shutil.rmtree(output_folder)
    
        # Copy new folder
        shutil.copytree(input_folder, output_folder)
    
    def __convert_ipynb_notebook(self, notebook):
        # Convert notebook
        body, resources = self.html_exporter.from_notebook_node(notebook)

        return body, resources
        
    def __write_output_file(self, outputdir, metadata, body, resource):
        
        # Get meta information
        title = metadata.get('title', 'Notes')

        context = {
            'html_body': body,
            'inline_css': resource['inlining']['css'],

            'title': title
        }

        # Get outputfile name
        filename = '{name}.html'.format(name='.'.join(metadata['filename'].split('.')[:-1]))
        output_filename = os.path.join(outputdir, filename)

        slug_title = metadata.get('slug', None)
        if slug_title is not None:
            output_filename = os.path.join(outputdir, '{name}.html'.format(name=slug_title))
        
        # Render notes
        self.__render_template(output_filename, self.__PATH_NOTE_TEMPLATE, context)

        return output_filename[output_filename.index('/') + 1: ]
    
    def __write_index_file(self, notes):
        context = {
            'notes': notes
        }

        # Render index.html
        self.__render_template(self.__OUTPUT_INDEX_FILE_PATH, self.__PATH_INDEX_TEMPLATE, context)
  
    def __render_template(self, output_path, template_path, context={}):
        # Read template
        with open(template_path, 'r') as file:
            template_data = file.read()
        
        # Render template
        template = Template(template_data)
        rendered_data = template.render(**context)

        # Added github static assets prefix
        rendered_data = self.__add_static_assets_prefix(rendered_data)

        # Write output file
        with open(output_path, 'w') as file:
            file.write(rendered_data)

    def __add_static_assets_prefix(self, html_data):
        data_url_regex = re.compile(r'data:image/.*')
        
        html = BeautifulSoup(html_data, 'html.parser')
        
        # Replace source of images
        for image in html.findAll('img'):
            
            # Skip data urls
            if data_url_regex.match(image['src']):
                continue

            # Make url absolute
            image['src'] = '/{}'.format(image['src']) if not image['src'].startswith('/') else image['src']
            image['src'] = '{}{}'.format(self.__STATIC_ASSETS_PREFIX, image['src'])
            
            # Add zoom.js attribute
            image['data-action'] = 'zoom'

        return str(html)

## Main

Main entry point of the process.

In [14]:
import time

class Main(object):

    @staticmethod
    def start():
        print('Cleaning directory...')
        Cleaner.clean()
        print('Creating HTML notes...')
        HTMLConverter().convert()
        print('Done!!!')

Main.start()

Cleaning directory...
Creating HTML notes...
Done!!!
